# Advent of Code 2018, Dyalog APL edition

To see a correct render of this notebook, check it out on [nbviewer](https://nbviewer.jupyter.org/github/xpqz/AoCDyalog/blob/master/Advent%20of%20Code%202018%20Dyalog%20APL.ipynb).

Annotated solutions in Dyalog APL.

Note that part of the charm of AoC is that every user (or at least groups of users) gets their own unique data set. Some of the solutions below exploit quirks in my particular data set, and so may conceivably not work for the general case.

In [2]:
⍝ Helper functions and common settings
⎕IO←0
assert←{⍺←'assertion failure' ⋄ 0∊⍵:⍺ ⎕signal 8 ⋄ shy←0}
lines←{⊃⎕NGET ⍵ 1}
ilines←{⍎¨lines ⍵}
line←{⊃lines ⍵}
sorted←{⍵[⍋⍵]}
sortbycol←⊢⌷⍨∘⊂∘⍒⌷⍤1
pairs←{↓(2÷⍨≢⍵) 2⍴⍵}
]box on -style=max -trains=tree -fns=on
]rows on

┌→─────────────────────────────────────┐
│Was ON -style=min -trains=tree -fns=on│
└──────────────────────────────────────┘

┌→──────┐
│Was OFF│
└───────┘

### Day 1: Chronal Calibration
https://adventofcode.com/2018/day/1

In [39]:
DAY1←ilines'data/2018/01.txt'

In [87]:
⊢Part1←+/DAY1
assert 477=Part1

477

For part 2 we're looking for the first frequency to recur. If we get to the end, take the last value and use that as the start value for the next iteration. We look for the first duplicate using Dyalog's [unique mask](https://aplwiki.com/wiki/Nub_Sieve) function (monadic `≠`), introduced in v18.

In [32]:
Day1p2←{t←⍬⋄0{t,←1↓+\⍺,⍵⋄1=≢e←t[⍸<\~≠t]:⊃e⋄⍵∇⍨⊃¯1↑t}⍵}

In [86]:
⊢Part2←Day1p2 DAY1
assert 390=Part2

390

### Day 2: Inventory Management System
https://adventofcode.com/2018/day/2

In [85]:
⊢Part1←×/+/⍉↑2 3∘∊¨(≢⊢)⌸¨DAY2←lines'data/2018/02.txt'
assert 8118=Part1

8118

In [84]:
⊢Part2←(=⌿l)/0⌷l←↑DAY2[⍸{1∊⍵}¨↓DAY2∘.{+/≠/⍺,⍪⍵}DAY2]
assert 'jbbenqtlaxhivmwyscjukztdp'≡Part2

┌→────────────────────────┐
│jbbenqtlaxhivmwyscjukztdp│
└─────────────────────────┘

### Day 3: No Matter How You Slice It
https://adventofcode.com/2018/day/3

In [128]:
DAY3←⍎¨¨'\d+'⎕s'&'¨lines'data/2018/03.txt'

In [152]:
R←1000 1000⍴0⋄_←{R[⍵]+←1}¨CL←{,((2⊃⍵)+⍳4⊃⍵)∘.,((1⊃⍵)+⍳3⌷⍵)}¨DAY3 ⍝ Convert origin + dx dy to coord pairs

In [153]:
⊢Part1←+/2≤∊R
assert 104126=Part1

104126

In [154]:
⊢Part2←1+⊃⍸{∧/1=R[⍵]}¨CL ⍝ Claim numbers are 1-indexed, so need to add 1
assert 695=Part2          

695

### Day 4: Repose Record
https://adventofcode.com/2018/day/4

Basically a series of set operations -- sort based on date, then split and merge into chunks based on each guard's log lines. Pair up into sleep-wake pairs and expand into the corresponding minute range. We can then use key `⌸` to find the most frequent minute for the guard with the most sleep minutes.

The regex

   '(?|.*(#\d+).*|.*:(\d\d)).*'
   
is a _branch reset_ `(?| ...|... )`, which means that the result of whichever of the two branches that matches can be referred to as `\1` in the replacement.

In [118]:
'iotag'⎕CY'dfns'
DAY4←sorted lines'data/2018/04.txt'

In [119]:
parsed←'(?|.*(#\d+).*|.*:(\d\d)).*'⎕r'\1'⊢DAY4 ⍝ Pick out guard number, and minutes for each sleep/wake item

In [120]:
guards←{0⊃0⊃⍵}¨merged←(≠{⍎1↓0⊃⍵}¨grouped)⊂grouped←sorted parsed⊂⍨'#'∘(1∊⍷)¨parsed

In [121]:
sleepiest←⊃⍒≢¨sleep←{∊{(⍎⍺) iotag ¯1+⍎⍵}/↑⍵}¨pairs¨{⊃,/{1↓⍵}¨⍵}¨merged

In [122]:
⊢Part1←(⍎1↓sleepiest⊃guards)×⊃((⊣/⊢⍤/⍨∘(⌈/=⊢)⊢/){⍺(≢⍵)}⌸)sleepiest⊃sleep
assert 138280=Part1

138280

For part 2, we seek the guard which has the highest minute sleep frequency. We have most of the parts already.

In [412]:
f←⊃⍒{⌈/1⌷⍉⍵}¨freq←({⍺,(≢⍵)}⌸)¨sleep

In [417]:
⊢Part2←(⍎1↓f⊃guards)×⊃(⊣/⊢⍤/⍨∘(⌈/=⊢)⊢/)f⊃freq
assert 89347=Part2

89347

### Day 5: Alchemical Reduction
https://adventofcode.com/2018/day/5

In [ ]:
DAY5←line'data/2018/05.txt'

In [418]:
r←∊'|'(1↓∘,,⍤0)p,⌽¨p←(⎕C⎕A),¨⎕A ⍝ Make the regex aA|Aa|bB|Bb ... etc

In [433]:
⊢Part1←≢{r⎕r''⊢⍵}⍣≡⊢DAY5        ⍝ Apply regex replace until no change
assert 9172=Part1

9172

In [435]:
⊢Part2←⌊/{≢{r⎕r''⊢⍵}⍣≡⊢(⍵⎕r''⍠'IC'1)DAY5}¨⎕A ⍝ This several minutes to run
assert 6550=Part2

6550

### Day 6: Chronal Coordinates
https://adventofcode.com/2018/day/6

This problem is very suited to an array language like APL. We create a 3D matrix where each major cell is a 2D matrix holding the manhattan distances from each point to the input coordinate. We can then process this along the z-axis to pick the layer index of the smallest distance. That gives us a "Nearest" map. What remains is to remove any "infinites", defined as the numbers which occur along any edge of the "Nearest" map, and finally find the highest frequency. 

The Python [solution](https://github.com/xpqz/aoc-18/blob/master/day6.py) is considerably longer and slower.

In [128]:
⎕io←0
DAY6←⍎¨lines'data/2018/06.txt'

In [129]:
(XMAX YMAX)←⊃↓⍉↑⌈/DAY6

For each of the 50 points, make an array where each value is the MHD from the point.

In [130]:
MHD←{+/|⍵-⍺}
Dist←{YMAX XMAX⍴⍵∘MHD¨,⍳YMAX XMAX}

Create the nearest distance map. A small hoop to jump through: points that are equidistant to several coordinates should be removed. We set those to ¯1.

In [131]:
Nearest←{v←⍋⍵⋄⍵[v[0]]=⍵[v[1]]:¯1⋄⊃v}¨↓[0]↑Dist¨⌽¨DAY6 ⍝ Flip each input pair so we have (y x)

Remove any items touching the boundaries of the array, as they're 'infinite'.

In [132]:
⊢Part1←1⊃0⌷1 sortbycol{⍺(≢⍵)}⌸(∊Nearest)~∪Nearest[⍸(⌽∨⊖)0∊¨⍳⍴Nearest]
assert 3989=Part1

3989

For part 2, we instead seek the set of points that each have a combined MHD to all coords < 10,000.

In [133]:
⊢Part2←+/{10000>+/⍵∘MHD¨DAY6}¨,⍳YMAX XMAX
assert 49715=Part2

49715

### Day 7: The Sum of Its Parts
https://adventofcode.com/2018/day/7

We have a graph which we need to traverse in a specific child-order - sorting the queue each iteration is a somewhat inefficient approach, but as the number of nodes is tiny it doesn't really matter.

In [123]:
DAY7←'^.+?([A-Z]).*([A-Z]).*$'⎕r'\1\2'⊢lines'data/2018/07.txt'

In [124]:
(KEY PRE)←↓⍉↑(↓⊃¨,∘⊂⌸⊢/¨)↓⌽↑DAY7 ⍝ Make the graph - splitting out nodes and their ascendants

In [125]:
Day7p1←{0=≢⍵:⍺⋄(⍺,p)∇(⍺,p)~⍨sorted ⍵∪{KEY/⍨⍵∘{∧/⍵∊⍺}¨PRE} ⍺,p←⊃⍵}

In [126]:
⊢Part1←⍬ Day7p1 sorted KEY~⍨∪∊DAY7
assert 'BHMOTUFLCPQKWINZVRXAJDSYEG'≡Part1

┌→─────────────────────────┐
│BHMOTUFLCPQKWINZVRXAJDSYEG│
└──────────────────────────┘

Part 2 -- process items concurrently in a worker pool of 5. Each job now takes 60 + a number corresponding to the item's position in the alphabet (A=60+1, B=60+2 etc). Replaces 200+ lines of my Python [solution](https://github.com/xpqz/aoc-18/blob/master/day7.py).

In [127]:
Workers←{0=slots←5-≢⍺:⍺⋄⍺,{⍵,⍨61+⎕A⍳⍵}¨(slots⌊≢⍵)↑⍵}

In [130]:
]dinput
Day7p2←{
    wrk←⍬ ⋄ sec←0
    ⍬ {
        (0=≢⍵)∧0=≢wrk:sec
        (remains jobs)←↓⍉↑wrk⊢←¯1∘+@0¨wrk Workers ⍵ ⍝ Fill any available slots and 'tick'
        wrk⊢←wrk/⍨0≠remains                         ⍝ Reap any dead worker processes
        sec+←1
        (⍺,sorted ready)∇(⍺,jobs)~⍨⍵∪{KEY/⍨⍵∘{∧/⍵∊⍺}¨PRE}⍺,ready←jobs/⍨0=remains
    } ⍵
}

In [131]:
⊢Part2←Day7p2 sorted KEY~⍨∪∊DAY7
assert 877=Part2

877

### Day 8: Memory Maneuver
https://adventofcode.com/2018/day/8

Release the recursions!

In [99]:
DAY8←⍎line'data/2018/08.txt'

In [100]:
get←{v←DAY8[CUR+⍳⍵]⋄CUR+←⍵⋄v} ⍝ Poor-man's generator

In [101]:
Day8p1←{(chc mtc)←get 2⋄ch←⍬ {⍵=0:⍺⋄(⍺,Day8p1⍬)∇⍵-1} chc⋄⊂ch (get mtc)}

In [102]:
CUR←0
⊢Part1←+/∊TREE←Day8p1⍬
assert 44838=Part1

44838

In [103]:
Day8p2←{(ch mt)←⍵⋄0=≢ch:+/mt⋄ind←¯1+mt~0⋄+/∇¨ch[(ind<≢ch)/ind]}

In [104]:
⊢Part2←Day8p2 ⊃TREE
assert 22198=Part2

22198

### Day 9: Marble Mania
https://adventofcode.com/2018/day/9

This will tax systems that are O(N) for vector insert or delete. We need a structure that's O(1) for both insert and delete for part 2 at least, here in the shape of a bi-directional linked list. Ugly, but does the job.

In [60]:
ELVES←430⍴0
CIRCLE←{⍵⊣⍵.(v l r pos start)←(,0) (,0) (,0) 0 0}⎕NS'' 

In [42]:
val←{⍵.pos⊃⍵.v}
rev7←{l←⍵ ⋄ {⍵=¯1:l ⋄ l.pos←l.pos⊃l.l ⋄ ∇ ⍵-1}7}
fwd←{⍵.pos←⍵.pos⊃⍵.r⋄⍵}
twenty3←{v←val rev7 CIRCLE⋄_←fwd del CIRCLE⋄ELVES[⍺]+←⍵+v⋄⍬}

In [43]:
]dinput
add←{
    ⍺.v,←⍵
    ⍺.l,←⍺.pos
    ⍺.r,←right←⍺.pos⊃⍺.r
    ⍺.pos←⍺.l[right]←⍺.r[⍺.pos]←¯1+≢⍺.v
    ⍺
}

In [44]:
]dinput
del←{
    left←⍵.pos⊃⍵.l
    ⍵.r[left]←right←⍵.pos⊃⍵.r
    ⍵.l[right]←left
    ⍵.pos=⍵.start:⊢⍵.pos←⍵.start←right
    ⍵.pos←right
    ⍵
}

In [61]:
]dinput
Day9←{
    max←⍵
    0 {
        max<⍵:⌈/ELVES
        0=23|⍵:((≢ELVES)|⍺+1)∇(⍵+1)⊣⍺ twenty3 ⍵
        _←fwd CIRCLE add ⍵
        ((≢ELVES)|⍺+1)∇⍵+1
    } 1
}

In [62]:
⊢Part1←Day9 71588
assert 422748=Part1

422748

For part 2, we're asked to increase the marble count by two orders of magnitude.

In [58]:
ELVES←430⍴0
CIRCLE←{⍵⊣⍵.(v l r pos start)←(,0) (,0) (,0) 0 0}⎕NS''

In [59]:
⎕PP←11                     ⍝ Up the precision limit for printing floats
⊢Part2←Day9 7158800        ⍝ Takes ~15s to run
assert 3412522480=Part2

3412522480

### Day 10: The Stars Align
https://adventofcode.com/2018/day/10

Back on solid array footing for day 10 - a task ideally suited for APL. Repeatedly add the velocity vector until the bounding box of the points no longer decreases.

In [74]:
⎕IO←0
DAY10←↑⍎¨¨'(-?\d+)'⎕s'\1'¨lines'data/2018/10.txt'

In [75]:
POS←,/DAY10[;0 1]⋄DELTA←,/DAY10[;2 3]

In [76]:
Bbox←{+/|-⌿2 2⍴(⌊⌿,⌈⌿)↑⍵} ⍝ Width + height of bounding box

In [77]:
MSG←DELTA-⍨DELTA∘{⍵+⍺}⍣{(Bbox ⍺)>Bbox ⍵}⊢POS ⍝ Add velocity until bounding box no longer decreases.

In [78]:
(W H)←|-⌿2 2⍴(⌊⌿,⌈⌿)↑MSG ⍝ Width and height of bounding box

In [79]:
⍉(-(W+1))↑⍉(-(H+1))↑'*'@(⊖¨MSG)⊢143 203⍴' ' ⍝ Prune our display: FPRBRRZA

┌→─────────────────────────────────────────────────────────────┐
↓****** ***** ***** ***** ***** ***** ****** ** │
│* * * * * * * * * * * * * * │
│* * * * * * * * * * * * * *│
│* * * * * * * * * * * * * *│
│***** ***** ***** ***** ***** ***** * * *│
│* * * * * * * * * * * ******│
│* * * * * * * * * * * * *│
│* * * * * * * * * * * * *│
│* * * * * * * * * * * * *│
│* * * * ***** * * * * ****** * *│
└──────────────────────────────────────────────────────────────┘

Part 2: how many iterations was that?

In [80]:
I←0
_←DELTA∘{⍵+⍺}⍣{I+←1⋄(Bbox ⍺)>Bbox ⍵}⊢POS

In [81]:
⊢Part2←I-1
assert 10027=Part2

10027

### Day 11: Chronal Charge
https://adventofcode.com/2018/day/11

More APL home court advantage here.

In [142]:
⎕IO←1        ⍝ Note: 1-based indexing
DAY11←7803

In [143]:
P←{¯5+1⊃10 10 10⊤rack×DAY11+⍺×rack←⍵+10} ⍝ The power function, as defined by the question

We apply the power function P to a 300×300 grid and then use stencil to define all 3×3 neighbourhoods which we then sum up. Note that stencil used this way creates neighbourhoods that straddle the boundaries, so we need to subtract 1,1 from the result. To be completely general we should probably have excluded these boundary neighbourhoods.

In [144]:
G←300 300⍴P/↑,⍳300 300
SUM←{+/,⍵}⌺3 3⊢G

In [145]:
MAX←⌈/,SUM

In [146]:
⊢Part1←¯1+⊖⊃⍸MAX⍷SUM ⍝ Question specifically asks for x,y so we flip the coordinate pair
assert 20 51≡Part1   ⍝ Submittable result format: 20,51

┌→────┐
│20 51│
└~────┘

Part 2 asks us to find the neighbourhood of any size that gives the largest sum. There are possibly smart ways to achieve this, noting that we could reuse smaller neighbourhood sums when calculating larger ones. But let's start with the simple way, looking for some probable patterns.

Let's consider all regions up to 50×50 -- reasonably quick:

In [147]:
G∘{⌈/,{+/,⍵}⌺⍵ ⍵⊢⍺}¨3+⍳50

┌→───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│36 55 58 70 73 75 76 99 101 117 117 116 119 125 122 113 110 96 74 81 73 71 63 64 57 62 55 43 13 6 2 ¯6 ¯32 ¯52 ¯72 ¯79 ¯101 ¯108 ¯117 ¯117 ¯117 ¯117 ¯127 ¯127 ¯136 ¯136 ¯146 ¯146 ¯151 ¯151│
└~───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘

We can see that the max sum (125) is at size 17, and then the values wobble downwards. So for part 2, we simply apply the same process as for part 1, but for a region size of 17×17, noting that we need to subtract 8,8 to skip the 17×17 neighbourhoods that aren't fully inside the boundary.

In [148]:
SUM←{+/,⍵}⌺17 17⊢G
MAX←⌈/,SUM
⊢Part2←¯8+⊖⊃⍸MAX⍷SUM  ⍝ Skip all neighbourhoods straddling a boundary
assert 230 272≡Part2  ⍝ Submittable result format: 230,272,17

┌→──────┐
│230 272│
└~──────┘

### Day 12: Subterranean Sustainability
https://adventofcode.com/2018/day/12

In [231]:
⎕IO←0
DAY12←'#'⎕R'1'¨'\.'⎕R'0'¨lines'data/2018/12.txt'

In [232]:
STATE←∊0 0,(⍎¨15↓0⊃DAY12),50⍴0 ⍝ Start state, as binary vector, padded a bit to the left and right
PATT←⍎¨¨5↑¨2↓DAY12             ⍝ Rule pattern bodies, as binary vectors
NEW←⍎¨¯1↑¨2↓DAY12              ⍝ Rule results as binary vector
RULES←NEW[⍋2⊥¨PATT]            ⍝ Sort rule results by the patterns converted from binary to decimal 0-31

The generational transform is now simply each overlapping binary 5-neighbourhood converted to decimal as indices into the RULES vector.

In [233]:
Next←{RULES[2⊥¨{⊂⍵}⌺5⊢⍵]}      ⍝ Transformation function

Part 1: run for 20 generations and sum up the pot-numbers for pots containing live plants. Note: not the number of live plants!

In [235]:
⊢Part1←+/¯2+⍸Next⍣20⊢STATE     ⍝ Remember to remove the left-side padding (¯2)
assert 3421=Part1

3421

Part 2: FIDDY BEEEEEEELLLION iterations! Obviously need to look at some data dependency trick here. Working hypothesis: check for emergent stable pattern. 

The two simplest potential options are:

1. Constant -- sum converges to constant value
2. Constant change -- sum _change_ converges to constant value

We can confirm that that the second case is what we have - let's look at the sums over time:

In [253]:
P2STATE←∊0 0,(⍎¨15↓0⊃DAY12),200⍴0         ⍝ Give ourselves a bit more growth room to the right
GENS←,+/¯2+⍸STATE
_←{v←Next ⍵⋄GENS,←+/¯2+⍸v⋄v}⍣150⊢P2STATE  ⍝ 150 generations

In [254]:
DIFFS←2-/GENS ⍝ Subtract current from previous; increases will show as negative

If we inspect how the generational differences change over the generations, we can see that after the 97th generation, every subsequent step adds 51 to the value:

In [255]:
97↓DIFFS

┌→──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│¯48 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51│
└~──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘

In [264]:
⎕PP←20
⊢Part2←(98⊃GENS)+51×50000000000-98
assert 2550000001195=Part2

2550000001195

### Day 13: Mine Cart Madness
https://adventofcode.com/2018/day/13

In [317]:
DAY13←↑lines'data/2018/13.txt'

Pick out the carts. 

We represent the compass bearings as 0, 1, 2, 3 for East, South, West and North respectively. The "turn state" for each cart defines its behaviour at intersections: left (0), straight (1), right (2).

In [318]:
⍝ row, col, heading, turn state
WEST ←(W←⍸'<'⍷DAY13),¨⊂0 0 1 2
SOUTH←(S←⍸'v'⍷DAY13),¨⊂1 0 1 2
EAST ←(E←⍸'>'⍷DAY13),¨⊂2 0 1 2
NORTH←(N←⍸'^'⍷DAY13),¨⊂3 0 1 2

"Fill in" the tiles the carts are sitting on. No cart is starting on an intersection or turn.

In [319]:
DAY13[W,E]←'-'
DAY13[N,S]←'|'

Part 1: what is the location of the first collision?
Part 2: each time we detect a collision, remove the two "wrecks" and carry on - what is the location of the last remaining cart at the moment of the final collision? 

As most of the bits are the same for part 1 and part 2, we can solve both at once.

In [339]:
]dinput
Day13←{
    graph←⍺
    loc←1@(↓(↑⍵)[;0 1])⊢150 150⍴0 ⍝ State: location of carts.
    Next←{
        dir←2⊃⍵ ⍝ Current bearing
        Rot←{∊(4 3⍴1 0 3 2 1 0 3 2 1 0 3 2)[0⊃⍵;1⊃⍵],1⊖1↓⍵} ⍝ Return new bearing and turn state
        
        Step←{
            ⍝ Move one step. Takes a left arg as a vector: dy dx turn1 turn2, and right arg a cart.
            ⍝ Move as defined by unit vector (dy, dx). Turn1 and turn2 are 
            ⍝ the new bearings when landing on a "/" or "\" respectively.
            ⍝ Thus: 0 ¯1 1 3 means "go one step east. If landing on "/", turn 
            ⍝ south. If "\", turn north.
            (dy dx slash backslash)←⍺
            loc[0⊃⍵;1⊃⍵]-←1
            pos←(dy+0⊃⍵),dx+1⊃⍵          ⍝ Step in stated direction
            loc[0⊃pos;1⊃pos]+←1
            tile←pos⌷graph               ⍝ Check the tile we landed on:
            '/'=tile:∊pos,slash,3↓⍵        ⍝ Turn: /
            '\'=tile:∊pos,backslash,3↓⍵    ⍝ Turn: \
            '+'=tile:∊pos,Rot 2↓⍵          ⍝ Intersection: +
            ∊pos,2↓⍵                       ⍝ Straight on: - or |
        }
        (dir⊃(0 ¯1 1 3)(1 0 0 2)(0 1 3 1)(¯1 0 2 0)) Step ⍵
    }

    crashes←⍬ ⍝ Store crash locations; only need first one for Part 1.
    Crash←{
        pos←2↑⍺
        crashes,←⊂pos
        loc[0⊃pos;1⊃pos]←0
        ⍵/⍨~(⊂pos)≡¨2↑¨⍵
    }
    MoveAll←{ ⍝ ⍺ are carts that have moved, ⍵ carts yet to move
        ⍺←⍬
        0=≢⍵:sorted ⍺
        cart←Next 0⊃⍵
        pos←2↑cart
        ⍝ If we have a 2 in the loc matrix, this cart collided with someone.
        2∊loc:(pos Crash ⍺)∇pos Crash 1↓⍵
        (⍺,⊂cart)∇1↓⍵
    }
    (⊂⊖0⊃crashes),⊂⊖2↑⊃MoveAll⍣{1=≢⍺}⊢⍵ ⍝ Repeat until only one remaining cart
}

In [340]:
⊢Solution←DAY13 Day13 WEST,SOUTH,EAST,NORTH
assert (76 108)(2 84)≡Solution

┌→────────────────┐
│ ┌→─────┐ ┌→───┐ │
│ │76 108│ │2 84│ │
│ └~─────┘ └~───┘ │
└∊────────────────┘

### Day 14: Chocolate Charts
https://adventofcode.com/2018/day/14

In [357]:
DAY14←765071

In [362]:
]dinput
Day14←{
    Mix←{new←10 10⊤+/⍺[⍵]⋄0≠0⊃new:new⋄1⊃new}
    data←3 7
    ⍵ {
        (≢data)≥⍺+10:⍺↓data
        data,←data Mix ⍵
        ⍺∇(≢data)|1+⍵+data[⍵]
    } 0 1
}

In [363]:
⊢Part1←Day14 DAY14
assert 3 1 7 1 1 2 3 9 2 3≡Part1

┌→──────────────────┐
│3 1 7 1 1 2 3 9 2 3│
└~──────────────────┘

Part 2: how many iterations before the given pattern appears to the left of us?